# Solar Panels and LST

Cross sectional analysis of solar panels in Tuas and housing estates in Singapore

In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import box
import numpy as np
import hvplot.pandas
import matplotlib.pyplot as plt
import requests

In [2]:
# Path to the GeoJSON file
geojson_path = r"C:\LocalOneDrive\Documents\Desktop\MTI\UHI-Project\MSE-ES-UHI\Data\boundaries.geojson"

# Read the GeoJSON file
postal_gdf = gpd.read_file(geojson_path)

print(postal_gdf)

        OBJECTID_1  FEA_ID                   BUILDING_NAME HOUSE_BLK_NO  \
0          9524876  172240                            None           27   
1          9524877  180932  KAMPONG GLAM CONSERVATION AREA          707   
2          9524878  180933  KAMPONG GLAM CONSERVATION AREA          705   
3          9524879  180934  KAMPONG GLAM CONSERVATION AREA          703   
4          9524880  181470                  FRANKEL ESTATE            6   
...            ...     ...                             ...          ...   
200342     9724937  836561                        AMBER 45           45   
200343     9724938  875786                            None           89   
200344     9724939  339812                            None          338   
200345     9724940  126196                            None          340   
200346     9724941  339814                            None          340   

                ROAD_NAME POSTAL_CODE DATA_TYPE_IND MULTI_ADDR_IND   SI_NO  \
0        JURONG PORT 

In [3]:
# Getting polygons

polygons_file_path = r"C:\LocalOneDrive\Documents\Desktop\MTI\UHI-Project\MSE-ES-UHI\Data\SG_geojson\SG.geojson"

polygons_gdf = gpd.read_file(polygons_file_path)

print(polygons_gdf)

            osm_id   osm_type           addr_street      building  \
0       1318498382      nodes                  None           yes   
1       1318498355      nodes              Libra Dr           yes   
2       1318498304      nodes            Jupiter Rd           yes   
3       4515515260      nodes  Serangoon Garden Way   residential   
4       6475355495      nodes                  None          roof   
...            ...        ...                   ...           ...   
357578    13648334  relations                  None           yes   
357579     7653043  relations                  None           yes   
357580    13648806  relations                  None           yes   
357581    13648807  relations                  None           yes   
357582    17333823  relations         Kelantan Road  construction   

                       name access_roof building_material addr_housenumber  \
0       Adelphi Peak Estate        None              None             None   
1              

In [4]:
def get_blk_no_gdf_road_name(road_name, building_name, postal_gdf):
    estate_gdf = postal_gdf[(postal_gdf['ROAD_NAME'] == road_name) & (postal_gdf['BUILDING_NAME'] == building_name)]
    return estate_gdf

def get_blk_no_gdf_name(building_name, postal_gdf):
    estate_gdf = postal_gdf[(postal_gdf['BUILDING_NAME'] == building_name)]
    return estate_gdf

In [5]:
def get_polygons_gdf(gdf, addr_street, polygons_gdf):
    house_blk_no_array = gdf['HOUSE_BLK_NO'].astype(str).tolist()
    house_blk_no_array = np.array(house_blk_no_array)
    filtered_polygons = polygons_gdf[(polygons_gdf['addr_street'].isin(addr_street)) &
                                     (polygons_gdf['addr_housenumber'].isin(house_blk_no_array))]
    estate_polygons_gdf = gpd.GeoDataFrame(
        filtered_polygons,
        geometry=filtered_polygons['geometry']
    )
    estate_polygons_gdf = estate_polygons_gdf[['geometry', 'addr_street', 'addr_housenumber']]
    estate_polygons_gdf = estate_polygons_gdf.to_crs('EPSG:4326')
    return estate_polygons_gdf

def get_polygons_gdf_blk_no_list(house_blk_no_array, addr_street, polygons_gdf):
    filtered_polygons = polygons_gdf[(polygons_gdf['addr_street'].isin(addr_street)) &
                                     (polygons_gdf['addr_housenumber'].isin(house_blk_no_array))]
    estate_polygons_gdf = gpd.GeoDataFrame(
        filtered_polygons,
        geometry=filtered_polygons['geometry']
    )
    estate_polygons_gdf = estate_polygons_gdf[['geometry', 'addr_street', 'addr_housenumber']]
    estate_polygons_gdf = estate_polygons_gdf.to_crs('EPSG:4326')
    return estate_polygons_gdf

In [6]:
def filter_estate_pixels(solar_estate_polygons, non_solar_estate_polygons, year_pixels_gdf):
    year_pixels_gdf = year_pixels_gdf.to_crs('EPSG:3857')
    year_pixels_gdf['geometry'] = year_pixels_gdf.geometry.apply(
        lambda p: box(p.x - 15, p.y - 15, p.x + 15, p.y + 15))
    
    all_polygons = gpd.GeoDataFrame(pd.concat([solar_estate_polygons, non_solar_estate_polygons], ignore_index=True))
    all_polygons = all_polygons.set_crs('EPSG:4326', inplace=False)
    print(all_polygons)
    # print(all_polygons)

    if all_polygons.crs != year_pixels_gdf.crs:
        all_polygons = all_polygons.to_crs(year_pixels_gdf.crs)
    
    comparison_pixels = gpd.sjoin(year_pixels_gdf, all_polygons, how="inner", op="intersects")

    if comparison_pixels.empty:
        raise ValueError("No pixels intersect with the polygons after spatial join.")
    
    return comparison_pixels, all_polygons

In [7]:
def filter_estate_pixels(solar_estate_polygons, non_solar_estate_polygons, year_pixels_gdf):
    # Ensure the pixels DataFrame is in the correct CRS
    year_pixels_gdf = year_pixels_gdf.to_crs('EPSG:3857')
    print("Pixels CRS after conversion:", year_pixels_gdf.crs)
    
    # Create 30m x 30m boxes around each point
    half_width = 15  # 15 meters half-width for a 30m box
    year_pixels_gdf['geometry'] = year_pixels_gdf.geometry.apply(
        lambda p: box(p.x - half_width, p.y - half_width, p.x + half_width, p.y + half_width))
    
    # Checking the min, max of the geometry bounds to ensure boxes are created correctly
    print("Pixel geometry bounds:", year_pixels_gdf.total_bounds)

    # Concatenate and set CRS for the polygons DataFrame
    all_polygons = gpd.GeoDataFrame(pd.concat([solar_estate_polygons, non_solar_estate_polygons], ignore_index=True))
    all_polygons = all_polygons.set_crs('EPSG:4326', inplace=False)
    print("Initial polygons CRS:", all_polygons.crs)

    # Transform all_polygons CRS to match year_pixels_gdf CRS if needed
    if all_polygons.crs != year_pixels_gdf.crs:
        all_polygons = all_polygons.to_crs(year_pixels_gdf.crs)
        print("Polygons CRS after conversion:", all_polygons.crs)

    # Check if the polygons are in the expected region after CRS transformation
    print("Polygon geometry bounds after CRS conversion:", all_polygons.total_bounds)

    # Perform spatial join to find intersecting pixels
    comparison_pixels = gpd.sjoin(year_pixels_gdf, all_polygons, how="inner", op="intersects")

    # Debugging: output the number of intersections found
    print("Number of intersecting pixels found:", len(comparison_pixels))

    if comparison_pixels.empty:
        print("Debug: No intersection found. Check the spatial extents and CRS settings.")
        raise ValueError("No pixels intersect with the polygons after spatial join.")
    
    return comparison_pixels, all_polygons

In [8]:
def plot_spatial_map(comparison_pixels, all_polygons, solar_estate_polygons):
    for df in [comparison_pixels, all_polygons, solar_estate_polygons]:
        print(df.columns)

    average_temps_by_block = comparison_pixels.groupby(['addr_housenumber'])['ST_B10_Celsius'].mean()

    min_temp = average_temps_by_block.min()
    max_temp = average_temps_by_block.max()

    # print(average_temps_by_block)
    
    all_polygons = all_polygons.to_crs('EPSG:4326')

    all_polygons.reset_index(drop=True, inplace=True)
    
    average_temps_by_block.rename('Average_Temp', inplace=True)  

    all_polygons = all_polygons.join(average_temps_by_block, on='addr_housenumber')

    all_polygons['Average_Temp'].fillna(0, inplace=True)

    # print(all_polygons)

    new_estate_avg = all_polygons[all_polygons['addr_housenumber'].isin(solar_estate_polygons['addr_housenumber'])]
    new_estate_avg = new_estate_avg['Average_Temp'].mean()
    print("Average LST of new estate: " + str(new_estate_avg))

    old_estate_avg = all_polygons[~all_polygons['addr_housenumber'].isin(solar_estate_polygons['addr_housenumber'])]
    old_estate_avg = old_estate_avg['Average_Temp'].mean()
    print("Average LST of old estate: " + str(old_estate_avg))
    
    plot_comparison = all_polygons.hvplot(
        geo=True,
        c='Average_Temp',
        cmap='oranges',
        clim=(min_temp, max_temp),
        hover_cols=['Average_Temp'],
        tiles='OSM',
        colorbar=True,
        frame_width=500,
        frame_height=500,
        line_color='black'
    )

    # Overlay for NorthArc polygons with white border and no fill
    northarc_borders = solar_estate_polygons.hvplot(
        geo=True,
        c='white',  # White color to match the requested border color
        color=None,  # No fill color, polygons will be transparent inside
        line_color='grey',  # Border color
        line_width=1,  # Border width
        frame_width=500,
        frame_height=500
    )

    combined_plot = plot_comparison * northarc_borders

    return combined_plot

In [9]:
cc30_2023 = pd.read_csv(r"C:\LocalOneDrive\Documents\Desktop\MTI\UHI-Project\MSE-ES-UHI\Data\FilteredData\2023_sg_excl_nature_cc30.csv")

# Convert the pandas DataFrame to a GeoDataFrame
cc30_2023_gdf = gpd.GeoDataFrame(cc30_2023, geometry=gpd.points_from_xy(cc30_2023.Longitude, cc30_2023.Latitude))

cc30_2023_gdf = cc30_2023_gdf.drop(['index_right'], axis=1)

cc30_2023_gdf = cc30_2023_gdf[cc30_2023_gdf['time'] == '2023-10-13']

In [10]:
cc30_2023_gdf['ST_B10_Celsius'] = cc30_2023_gdf['ST_B10']  * 0.00341802 + 149 - 273.15

# Scale and offset values for SR_B4 (Near Infrared) and SR_B5 (Shortwave Infrared 1)
scale = 2.75e-05
offset = -0.2

# Apply scale and offset to the bands
cc30_2023_gdf['SR_B4_Scaled'] = (cc30_2023_gdf['SR_B4'] * scale) + offset
cc30_2023_gdf['SR_B5_Scaled'] = (cc30_2023_gdf['SR_B5'] * scale) + offset

cc30_2023_gdf['NDVI'] = (cc30_2023_gdf['SR_B5_Scaled'] - cc30_2023_gdf['SR_B4_Scaled']) / (cc30_2023_gdf['SR_B5_Scaled'] + cc30_2023_gdf['SR_B4_Scaled'])

# # Define the filtering values if filtering is required
# qa_pixel_filter_1 = [5504, 22280, 5896, 7440, 23826, 56854, 24082, 56598, 7698, 7442, 54596, 24144, 23888, 56916]
# # sr_qa_aerosol_filter_1 = [192, 194, 196, 224, 228]
# sr_qa_aerosol_filter_2 = [192, 194, 196]

# # Filter by 'QA_PIXEL' values
# cc30_2023_gdf = cc30_2023_gdf[~cc30_2023_gdf['QA_PIXEL'].isin(qa_pixel_filter_1)]

# # Filter by 'SR_QA_AEROSOL' values
# cc30_2023_gdf = cc30_2023_gdf[~cc30_2023_gdf['SR_QA_AEROSOL'].isin(sr_qa_aerosol_filter_2)]

# Setting CRS for plotting if not already set
if cc30_2023_gdf.crs is None:
    cc30_2023_gdf.set_crs('EPSG:4326', inplace=True)

## Toll Logistics Building

In [10]:
cc30_2023_clementi_gdf = cc30_2023_gdf[cc30_2023_gdf['name'] == 'CLEMENTI']

In [7]:
toll_gdf = get_blk_no_gdf_name('LOGIS HUB @ CLEMENTI', postal_gdf)

In [11]:
toll_polygons = get_polygons_gdf(toll_gdf, ['Clementi Loop'], polygons_gdf)

In [15]:
west_logis_hub_gdf = get_blk_no_gdf_name('CLEMENTI WEST LOGISPARK', postal_gdf)

In [16]:
west_logis_hub_polyons = get_polygons_gdf(west_logis_hub_gdf, ['Clementi Loop'], polygons_gdf)

In [19]:
eac_building_gdf = get_blk_no_gdf_name('EAC BUILDING', postal_gdf)

In [20]:
eac_polygons = get_polygons_gdf(eac_building_gdf, ['Clementi Loop'], polygons_gdf)

In [22]:
clementi_distribution_gdf = get_blk_no_gdf_name('CLEMENTI DISTRIBUTION CENTRE', postal_gdf)

In [23]:
clementi_distribution_polygons = get_polygons_gdf(clementi_distribution_gdf, ['Clementi Loop'], polygons_gdf)

In [27]:
non_solar_clementi_polygons = pd.concat([west_logis_hub_polyons, eac_polygons, clementi_distribution_polygons])

In [31]:
comparison_pixels, all_polygons = filter_estate_pixels(toll_polygons, non_solar_clementi_polygons, cc30_2023_clementi_gdf)

c:\Users\teomi\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3508: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [34]:
plot_spatial_map(comparison_pixels, all_polygons, toll_polygons)

Average LST of new estate: 46.40626826857145
Average LST of old estate: 46.25375486046328


:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Polygons.I  :Polygons   [Longitude,Latitude]   (Average_Temp)
   .Polygons.II :Polygons   [Longitude,Latitude]

## Tuas Bay Lane

In [446]:
cc30_2023_tuas_gdf = cc30_2023_gdf[cc30_2023_gdf['name'] == 'TUAS']

cc30_2023_tuas_gdf = cc30_2023_tuas_gdf.set_crs('EPSG:4326', allow_override=True)

In [48]:
def get_node_coordinates(node_ids):
    url = "http://overpass-api.de/api/interpreter"
    query = f"""
    [out:json];
    (node(id:{','.join(map(str, node_ids))}););
    out;
    """
    response = requests.get(url, params={'data': query})
    data = response.json()
    coordinates = [{'lon': node['lon'], 'lat': node['lat']} for node in data['elements']]
    return coordinates

def get_polygons_from_regions(regions):
    polygons_data = []

    for region, nodes in regions.items():
        coordinates = get_node_coordinates(nodes)
        # Create points directly in GeoDataFrame constructor
        points = gpd.points_from_xy([coord['lon'] for coord in coordinates], [coord['lat'] for coord in coordinates])
        gdf = gpd.GeoDataFrame(coordinates, geometry=points)
        polygon = gpd.GeoDataFrame({'region': [region], 'geometry': [gdf.unary_union.convex_hull]})

        polygons_data.append(polygon)

    # Concatenate all regional polygons into a single GeoDataFrame
    all_polygons_gdf = pd.concat(polygons_data)

    return all_polygons_gdf

In [507]:
tuas_solar_regions = {
    "Tuas Bay Lane Solar Panel 1": [
    10728873889,
    10728873888,
    10728873887,
    10728873886,
    10728873885,
    10728873884,
    10728873883,
    10728873882,
    10728873881,
    10728873880,
    10728873879,
    10728873878,
    10728873877,
    10728873889 
    ], 

    "Tuas Bay Lane Solar Panel 2": [
        10728873870,
        10728873869,
        10728873868,
        10728873867,
        10728873866,
        10728873865,
        10728873870
    ], 

    "Tuas Bay Lane Solar Panel 3": [
        10728873876,
        10728873875,
        10728873874,
        10728873873,
        10728873872,
        10728873871,
        10728873876
    ], 

    "Tuas Bay Lane Solar Panel 4": [
        5522837717,
        5522837714,
        5522837691,
        5522837688,
        6553437317,
        5522837689,
        5522837690,
        5522837686,
        5522837687,
        5522837685,
        5522837684,
        5522837715,
        5522837716,
        5522837680,
        5522837679,
        5522837678,
        5522837681,
        5522837682,
        5522837676,
        6553437316,
        5522837677,
        5522837683,
        5522837717 
    ], 

    "Tuas Bay Lane Solar Panel 5": [
        5522837698,
        5522837707,
        5522837692,
        5522837708,
        5522837705,
        5522837696,
        5522837706,
        5522837697,
        6553437322,
        5522837699,
        5522837704,
        5522837712,
        5522837703,
        5522837711,
        5522837713,
        5522837701,
        5522837710,
        5522837709,
        5522837702,
        5522837700,
        6553437323,
        5522837698 
    ]
}

In [293]:
tuas_solar_regions = {
    "Kim Soon Lee Pte Ltd": [
        1326022457,
        1326022388,
        1326022442,
        1326022444,
        1326022457
    ]
}

In [508]:
tuas_bay_lane_solar_polygons = get_polygons_from_regions(tuas_solar_regions)

tuas_bay_lane_solar_polygons = tuas_bay_lane_solar_polygons.set_crs('EPSG:4326', allow_override=True)

In [509]:
non_solar_buildings = {
    "Tuas Bay Lane Non-Solar Building 1": [
        2584494600,
        2584494595,
        2584494609,
        2584494632,
        2584494628,
        2584494665,
        2584494642,
        2584494600
    ], 

    "Novartis Biomanufacturing Plant 1": [
        2584494335,
        2584494370,
        2584494344,
        2584494420,
        2584494375,
        2584494271,
        2584494284,
        2584494302,
        5266522653,
        2584494335 
    ], 

    "Novartis Biomanufacturing Plant 2": [
        2584494288,
        2584494277,
        2584494267,
        2584494279,
        2584494288
    ],

    "Novartis Biomanufacturing Plant 3": [
        2584494442,
        2584494431,
        2584494424,
        2584494432,
        2584494442
    ], 

    "Novartis Biomanufacturing Plant 3": [
        2584494450,
        2584494455,
        2584494444,
        2584494435,
        2584494450
    ],

    "Novartis Biomanufacturing Plant 4": [
        2584494445,
        2584494441,
        2584494426,
        2584494440,
        2584494445
    ],

    "Tuas Bay Lane Non-Solar Building 2": [
        5522838806,
        5522838803,
        5522838804,
        5522838805,
        5522838806
    ],

    "Tuas Bay Lane Non-Solar Building 3": [
        5522838810,
        5522838808,
        5522838807,
        5522838802,
        5522838809,
        5522838812,
        5522838811,
        5522838810
    ],

    "Tuas Bay Lane Non-Solar Building 4": [
        5522838816,
        5522838815,
        5522838814,
        5522838813,
        5522838816
    ], 

    "Tuas Bay Lane Non-Solar Building 5": [
        5522837667,
        5522838817,
        5522838818,
        5522837669,
        5522837667
    ], 

    "Tuas Bay Lane Non-Solar Building 6": [
        5522837672,
        5522837671,
        5522837670,
        5522837667,
        5522837669,
        5522837668,
        5522837673,
        5522837672
    ], 

    "Tuas Bay Lane Non-Solar Building 7": [
        5522837675,
        5522837672,
        5522837673,
        5522837674,
        5522837675
    ],

    "Tuas Bay Lane Non-Solar Building 8": [
        5522837696,
        5522837695,
        5522837694,
        5522837693,
        5522837715,
        5522837716,
        5522837692,
        5522837708,
        5522837705,
        5522837696
    ], 

    "Pfizer 1": [
        5268270602,
        4501872217,
        4501872233,
        4501872249,
        4501872247,
        5268270599,
        4501872254,
        5268270600,
        4501872252,
        4501872260,
        4501872242,
        4501872241,
        4501872237,
        5268270601,
        5268270602
    ], 

    "Pfizer 2": [
        10728873837,
        10728873836,
        10728873835,
        10728873834,
        10728873837
    ], 

    "Pfizer 3": [
        4501872255,
        4501872258,
        4501872256,
        4501872266,
        4501872273,
        4501872268,
        4501872270,
        4501872265,
        4501872261,
        4501872263,
        4501872255
    ], 

    "Pfizer 4": [
        4501872291,
        4501872289,
        4501872277,
        4501872288,
        4501872291
    ], 

    "Pfizer 5": [
        4501872267,
        4501872264,
        4501872250,
        4501872257,
        4501872267
    ], 

    "Pfizer 6": [
        4501872303,
        4501872301,
        4501872275,
        4501872287,
        4501872305,
        4501872307,
        4501872313,
        4501872308,
        4501872303
    ], 

    "Pfizer 7": [
        4501872326,
        4501872323,
        4501872321,
        4501872318,
        4501872309,
        4501872310,
        4501872292,
        4501872295,
        4501872296,
        4501872298,
        4501872300,
        4501872299,
        4501872315,
        4501872316,
        4501872319,
        4501872320,
        4501872326
    ], 

    "Pfizer 8": [
        4501872286,
        4501872283,
        4501872271,
        4501872276,
        4501872286
    ], 

    "Pfizer 9": [
        4501872281,
        4501872279,
        4501872269,
        4501872272,
        4501872281
    ],

    "Pfizer 10": [
        4501872297,
        4501872294,
        4501872290,
        4501872293,
        4501872297
    ], 

    "Pfizer 11": [
        4501872311,
        4501872306,
        4501872302,
        4501872304,
        4501872311
    ], 

    "Pfizer 12": [
        4501872324,
        4501872314,
        4501872312,
        4501872317,
        4501872324
    ], 

    "Pfizer 13": [
        4501872329,
        4501872328,
        4501872322,
        4501872325,
        4501872329
    ],

    "Pfizer 14": [
        4501872331,
        4501872339,
        4501872337,
        4501872327,
        4501872331
    ], 

    "Pfizer 15": [ 
        4501872338,
        4501872335,
        4501872330,
        4501872332,
        4501872338
    ], 

    "Pfizer 16": [
        4501872334,
        4501872333,
        4501872336,
        4501872342,
        4501872334
    ], 

    "Pfizer 17": [ 
        4501872344,
        4501872343,
        4501872340,
        4501872341,
        4501872344
    ]
}

# non_solar_blk_nos = ['10', '12', '14', '16', '18', '20', '22', '26', 
#                      '28', '30', '32', '34', '36', '38', '40', '42', 
#                      '17', '19', '21', '23', '25', '27', '29', '31',
#                      '33', '35', '37', '39', '41', '43', '45', '47', 
#                      '49', '51', '53', '55']

In [510]:
# west_link_one_gdf = get_blk_no_gdf_name('WESTLINK ONE', postal_gdf)
# west_link_two_gdf = get_blk_no_gdf_name('WESTLINK TWO', postal_gdf)
# enterprise_logistics_centre_gdf = get_blk_no_gdf_name('ENTERPRISE LOGISTICS CENTRE', postal_gdf)

# west_link_one_polygons = get_polygons_gdf(west_link_one_gdf, ['Tuas View Place'], polygons_gdf)
# west_link_two_polygons = get_polygons_gdf(west_link_two_gdf, ['Tuas View Place'], polygons_gdf)
# enterprise_logistics_centre_polygons = get_polygons_gdf(enterprise_logistics_centre_gdf, ['Tuas View Place'], polygons_gdf)
# tuas_view_place_polygons = get_polygons_gdf_blk_no_list(non_solar_blk_nos, ['Tuas View Place'], polygons_gdf)

tuas_bay_lane_non_solar_polygons = gpd.GeoDataFrame(get_polygons_from_regions(non_solar_buildings))

tuas_bay_lane_non_solar_polygons

,region,geometry
0,Tuas Bay Lane Non-Solar Building 1,"POLYGON ((103.62945 1.29898, 103.62910 1.29911..."
0,Novartis Biomanufacturing Plant 1,"POLYGON ((103.63636 1.29527, 103.63617 1.29533..."
0,Novartis Biomanufacturing Plant 2,"POLYGON ((103.63602 1.29521, 103.63573 1.29532..."
0,Novartis Biomanufacturing Plant 3,"POLYGON ((103.63534 1.29649, 103.63493 1.29664..."
0,Novartis Biomanufacturing Plant 4,"POLYGON ((103.63516 1.29641, 103.63490 1.29651..."
0,Tuas Bay Lane Non-Solar Building 2,"POLYGON ((103.63818 1.29742, 103.63789 1.29753..."
0,Tuas Bay Lane Non-Solar Building 3,"POLYGON ((103.63832 1.29784, 103.63820 1.29788..."
0,Tuas Bay Lane Non-Solar Building 4,"POLYGON ((103.63739 1.29817, 103.63709 1.29829..."
0,Tuas Bay Lane Non-Solar Building 5,"POLYGON ((103.63701 1.29834, 103.63687 1.29839..."
0,Tuas Bay Lane Non-Solar Building 6,"POLYGON ((103.63687 1.29839, 103.63674 1.29844..."


In [511]:
tuas_bay_lane_solar_polygons['addr_housenumber'] = tuas_bay_lane_solar_polygons['region']
tuas_bay_lane_non_solar_polygons['addr_housenumber'] = tuas_bay_lane_non_solar_polygons['region']

In [512]:
comparison_pixels, all_polygons = filter_estate_pixels(tuas_bay_lane_solar_polygons, tuas_bay_lane_non_solar_polygons, cc30_2023_tuas_gdf)

Pixels CRS after conversion: EPSG:3857
Pixel geometry bounds: [11533327.48572822   134772.80710934 11540226.08181298   150634.29229034]
Initial polygons CRS: EPSG:4326
Polygons CRS after conversion: EPSG:3857
Polygon geometry bounds after CRS conversion: [11535938.64336522   144194.35461244 11536979.12438177   145010.98158755]
Number of intersecting pixels found: 341


c:\Users\teomi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\array.py:1470: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
c:\Users\teomi\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3508: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [330]:
plot_spatial_map(comparison_pixels, all_polygons, tuas_bay_lane_solar_polygons)

Index(['Unnamed: 0', 'Longitude', 'Latitude', 'SR_B1', 'SR_B2', 'SR_B3',
       'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_QA_AEROSOL', 'ST_B10',
       'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA',
       'ST_TRAD', 'ST_URAD', 'QA_PIXEL', 'QA_RADSAT', 'geometry', 'name',
       'time', 'ST_B10_Celsius', 'SR_B4_Scaled', 'SR_B5_Scaled', 'NDVI',
       'index_right', 'region', 'addr_housenumber'],
      dtype='object')
Index(['region', 'geometry', 'addr_housenumber'], dtype='object')
Index(['region', 'geometry', 'addr_housenumber'], dtype='object')
Average LST of new estate: nan
Average LST of old estate: 47.379987052448286


:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Polygons.I  :Polygons   [Longitude,Latitude]   (Average_Temp)
   .Polygons.II :Polygons   [Longitude,Latitude]

## Kim Soon Lee Building (-)

In [392]:
ksl_solar_blk_nos = ['51', '4', '6']

ksl_solar_polygons = get_polygons_gdf_blk_no_list(ksl_solar_blk_nos, ['Tuas West Street'], polygons_gdf)

ksl_solar_regions = {
    "Maha Chemical": [
        1325161165,
        1325161183,
        1325161163,
        1325161140,
        1325161155
    ]
}

In [393]:
maha_chemical_polygon = get_polygons_from_regions(ksl_solar_regions)
maha_chemical_polygon['addr_housenumber'] = maha_chemical_polygon['region']

ksl_solar_polygons = ksl_solar_polygons.append(maha_chemical_polygon)

C:\Users\teomi\AppData\Local\Temp\ipykernel_29748\4250159168.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ksl_solar_polygons = ksl_solar_polygons.append(maha_chemical_polygon)
c:\Users\teomi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\array.py:1470: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))


In [400]:
ksl_non_solar_blk_nos = ['32', '8', '2', '18', '1', '3', '5', '7', '9', '151', '153', '155', '157', '159', '161', '32']

ksl_non_solar_road_names = ['Tuas West Road', 'Tuas West Aveneue', 'Pioneer Road', 'Tuas West Street']

ksl_non_solar_polygons = get_polygons_gdf_blk_no_list(ksl_non_solar_blk_nos, ksl_non_solar_road_names, polygons_gdf)

In [399]:
ksl_non_solar_regions = {
    "B1": [
        1325148642,
        1325148578,
        1325148579,
        1325148547,
        1325148642
    ], 

    "B2": [
        1325148575,
        1325148544,
        1325148648,
        1325148590,
        1325148575
    ], 

    "B3": [
        1325148549,
        1325148580,
        1325148649,
        1325148629,
        1325148549
    ],

    "B4": [
        5270827633,
        5270827632,
        5270827631,
        5270827630,
        5270827629,
        5270827628,
        5270827627,
        5270827626,
        5270827625,
        5270827633
    ], 
    
    "32": [
        1325161140,
        1325161163,
        1325161180,
        2584496747,
        2584496767,
        1325161157,
        1325161140
    ], 

    "30": [
        1325161164,
        1325161136,
        1325161146,
        1325161177,
        1325161164
    ], 

    "28": [
        1325161143,
        1325161164,
        1325161177,
        1325161167,
        1325161143
    ], 

    "20": [
        1325148543,
        1325148576,
        1325148546,
        1325148641,
        1325148640,
        1325148516,
        1325148566,
        1325148563,
        1325148543
    ], 

    "18": [
        1325148637,
        1325148638,
        1325148613,
        1325148582,
        1325148637
    ], 

    "151": [
        1325148617,
        1325148512,
        1325148511,
        1325148503,
        1325148617
    ], 

    "153": [
        1325148512,
        1325148617,
        1325148562,
        1325148595,
        1325148512
    ], 

    "155": [
        1325148591,
        1325148564,
        1325148531,
        1325148608,
        1325148591
    ], 

    "157": [
        1325148564,
        1325148591,
        1325148502,
        1325148646,
        1325148564
    ],

    "159": [
        1325148510,
        1325148513,
        1325148514,
        1325148526,
        1325148510 
    ], 

    "161": [
        1325148510,
        1325148526,
        1325148527,
        1325148607,
        1325148510
    ]
}

In [401]:
additional_non_solar_polygons = get_polygons_from_regions(ksl_non_solar_regions)

additional_non_solar_polygons['addr_housenumber'] = additional_non_solar_polygons['region']

ksl_non_solar_polygons = ksl_non_solar_polygons.append(additional_non_solar_polygons)

C:\Users\teomi\AppData\Local\Temp\ipykernel_29748\1149671267.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ksl_non_solar_polygons = ksl_non_solar_polygons.append(additional_non_solar_polygons)
c:\Users\teomi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\array.py:1470: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))


In [402]:
comparison_pixels, all_polygons = filter_estate_pixels(ksl_solar_polygons, ksl_non_solar_polygons, cc30_2023_tuas_gdf)

c:\Users\teomi\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3508: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [403]:
plot_spatial_map(comparison_pixels, all_polygons, ksl_solar_polygons)

Index(['Unnamed: 0', 'Longitude', 'Latitude', 'SR_B1', 'SR_B2', 'SR_B3',
       'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_QA_AEROSOL', 'ST_B10',
       'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA',
       'ST_TRAD', 'ST_URAD', 'QA_PIXEL', 'QA_RADSAT', 'geometry', 'name',
       'time', 'ST_B10_Celsius', 'SR_B4_Scaled', 'SR_B5_Scaled', 'NDVI',
       'index_right', 'addr_street', 'addr_housenumber', 'region'],
      dtype='object')
Index(['geometry', 'addr_street', 'addr_housenumber', 'region'], dtype='object')
Index(['geometry', 'addr_street', 'addr_housenumber', 'region'], dtype='object')
Average LST of new estate: 53.71032608250001
Average LST of old estate: 52.51324558901301


:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Polygons.I  :Polygons   [Longitude,Latitude]   (Average_Temp)
   .Polygons.II :Polygons   [Longitude,Latitude]

## Mercedes Logistics Centre

In [460]:
mercedes_solar_regions = {
    "315": [
        1325392084,
        1325392104,
        5270483721,
        5270483722,
        1325392052,
        5270483723,
        5270483724,
        5270483725,
        5270483727,
        5270483726,
        5270483728,
        1325392130,
        1325392084
    ], 

    # "52": [
    #     1325392099,
    #     1325392039,
    #     2584496647,
    #     2584496640,
    #     1325392132,
    #     1325392050,
    #     1325392099
    # ], 

    # "Panasonic": [
    #     1325392073,
    #     1325392070,
    #     1325392098,
    #     5215169619,
    #     5215170621,
    #     1325392035,
    #     1325392125,
    #     1325392060,
    #     1325392087,
    #     5215170625,
    #     5215170624,
    #     5215169620,
    #     5215169618,
    #     5215170623,
    #     5215170622,
    #     1325392116,
    #     1325392073
    # ], 

    "50": [
        1325392094,
        1325392093,
        1325392074,
        1325392069,
        1325392094
    ]
}

In [461]:
mercedes_non_solar_regions = {
    "Mercedes": [
        1325392115,
        1325392044,
        5266907593,
        5266907592,
        1325392048,
        1325392042,
        1325392115
    ]
}

In [462]:
mercedes_solar_polygons = get_polygons_from_regions(mercedes_solar_regions)
mercedes_non_solar_polygons = get_polygons_from_regions(mercedes_non_solar_regions)

panasonic_polygon = polygons_gdf[(polygons_gdf['name'] == 'Panasonic') & (polygons_gdf['addr_housenumber'] == '285')]
b52_polygon = polygons_gdf[(polygons_gdf['addr_street'] == 'Tuas Avenue 1') & (polygons_gdf['addr_housenumber'] == '52')]

mercedes_solar_polygons['addr_housenumber'] = mercedes_solar_polygons['region']
mercedes_non_solar_polygons['addr_housenumber'] = mercedes_non_solar_polygons['region']

mercedes_solar_polygons = mercedes_solar_polygons.append(panasonic_polygon)
mercedes_solar_polygons = mercedes_solar_polygons.append(b52_polygon)

C:\Users\teomi\AppData\Local\Temp\ipykernel_29748\919289726.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mercedes_solar_polygons = mercedes_solar_polygons.append(panasonic_polygon)
c:\Users\teomi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\array.py:1470: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
C:\Users\teomi\AppData\Local\Temp\ipykernel_29748\919289726.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mercedes_solar_polygons = mercedes_solar_polygons.append(b52_polygon)


In [463]:
comparison_pixels, all_polygons = filter_estate_pixels(mercedes_solar_polygons, mercedes_non_solar_polygons, cc30_2023_tuas_gdf)

Pixels CRS after conversion: EPSG:3857
Pixel geometry bounds: [11533327.48572822   134772.80710934 11540226.08181298   150634.29229034]
Initial polygons CRS: EPSG:4326
Polygons CRS after conversion: EPSG:3857
Polygon geometry bounds after CRS conversion: [11538526.43190795   147836.10176314 11538957.4609763    148271.32235335]
Number of intersecting pixels found: 117


c:\Users\teomi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\array.py:1470: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
c:\Users\teomi\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3508: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [464]:
plot_spatial_map(comparison_pixels, all_polygons, mercedes_solar_polygons)

Index(['Unnamed: 0', 'Longitude', 'Latitude', 'SR_B1', 'SR_B2', 'SR_B3',
       'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_QA_AEROSOL', 'ST_B10',
       'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA',
       'ST_TRAD', 'ST_URAD', 'QA_PIXEL', 'QA_RADSAT', 'geometry', 'name_left',
       'time', 'ST_B10_Celsius', 'SR_B4_Scaled', 'SR_B5_Scaled', 'NDVI',
       'index_right', 'region', 'addr_housenumber', 'osm_id', 'osm_type',
       'addr_street', 'building', 'name_right', 'access_roof',
       'building_material', 'roof_material'],
      dtype='object')
Index(['region', 'geometry', 'addr_housenumber', 'osm_id', 'osm_type',
       'addr_street', 'building', 'name', 'access_roof', 'building_material',
       'roof_material'],
      dtype='object')
Index(['region', 'geometry', 'addr_housenumber', 'osm_id', 'osm_type',
       'addr_street', 'building', 'name', 'access_roof', 'building_material',
       'roof_material'],
      dtype='object')
Average LST of new estate: 50.75637

:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Polygons.I  :Polygons   [Longitude,Latitude]   (Average_Temp)
   .Polygons.II :Polygons   [Longitude,Latitude]

## Quantum Solutions

In [497]:
qs_solar_polygon = get_polygons_gdf_blk_no_list(['20'], ['Gul Way'], polygons_gdf)

In [498]:
qs_non_solar_polygons = get_polygons_gdf_blk_no_list(['21', '23', '25', '27', '29', '31', '33', '35', '37'], ['Gul Avenue'], polygons_gdf)

In [499]:
cc30_2023_pioneer_gdf = cc30_2023_gdf[cc30_2023_gdf['name'] == 'PIONEER']

In [500]:
comparison_pixels, all_polygons = filter_estate_pixels(qs_solar_polygon, qs_non_solar_polygons, cc30_2023_pioneer_gdf)

Pixels CRS after conversion: EPSG:3857
Pixel geometry bounds: [11538246.89338713   143475.52314359 11543437.85671713   148189.17854621]
Initial polygons CRS: EPSG:4326
Polygons CRS after conversion: EPSG:3857
Polygon geometry bounds after CRS conversion: [11540156.73924647   146080.68412516 11540551.77872344   146376.5488843 ]
Number of intersecting pixels found: 132


c:\Users\teomi\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3508: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [501]:
plot_spatial_map(comparison_pixels, all_polygons, qs_solar_polygon)

Index(['Unnamed: 0', 'Longitude', 'Latitude', 'SR_B1', 'SR_B2', 'SR_B3',
       'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_QA_AEROSOL', 'ST_B10',
       'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA',
       'ST_TRAD', 'ST_URAD', 'QA_PIXEL', 'QA_RADSAT', 'geometry', 'name',
       'time', 'ST_B10_Celsius', 'SR_B4_Scaled', 'SR_B5_Scaled', 'NDVI',
       'index_right', 'addr_street', 'addr_housenumber'],
      dtype='object')
Index(['geometry', 'addr_street', 'addr_housenumber'], dtype='object')
Index(['geometry', 'addr_street', 'addr_housenumber'], dtype='object')
Average LST of new estate: 50.10913243830987
Average LST of old estate: 51.27892539223018


:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Polygons.I  :Polygons   [Longitude,Latitude]   (Average_Temp)
   .Polygons.II :Polygons   [Longitude,Latitude]

## Dalwin Marine/Tuas South Avenue 1

In [503]:
dalwin_marine_solar_regions = {
    "Dorm 1": [
        2584496621,
        2584496610,
        2584496613,
        2584496622,
        2584496621
    ], 

    "Dorm 2": [
        2584496589,
        2584496577,
        2584496583,
        2584496591,
        2584496589
    ], 

    "Dorm 3": [
        2584496573,
        2584496579,
        2584496618,
        2584496616,
        2584496573
    ], 

    "Dorm 4": [
        2584496563,
        2584496548,
        2584496539,
        2584496552,
        2584496563
    ], 

    "Dorm 5": [
        2584496578,
        2584496581,
        2584496585,
        2584496580,
        2584496574,
        2584496570,
        2584496557,
        2584496566,
        2584496572,
        2584496575,
        2584496578
    ], 

    "Dorm 6": [
        2584496587,
        2584496590,
        2584496586,
        2584496582,
        2584496587
    ],

    "Dorm 7": [
        2584496592,
        2584496595,
        2584496597,
        2584496605,
        2584496593,
        2584496588,
        2584496592
    ]
}

In [502]:
dalwin_marine_non_solar_regions = {
    "Block 1": [ 
        1326043710,
        1326043943,
        1326043958,
        1326043647,
        1326043833,
        1326043813,
        1326043710
    ],

    "Block 2": [ 
        1326043888,
        1326043612,
        1326043920,
        1326043835,
        1326043810,
        1326043816,
        1326043888
    ],

    # "Block 3": [
    #     1326043867,
    #     1326043825,
    #     1326043990,
    #     1326043718,
    #     1326043867
    # ]   
}

In [504]:
dalwin_marine_solar_polygons = gpd.GeoDataFrame(get_polygons_from_regions(dalwin_marine_solar_regions))
dalwin_marine_non_solar_polygons = gpd.GeoDataFrame(get_polygons_from_regions(dalwin_marine_non_solar_regions))

dalwin_marine_solar_polygons.set_crs('EPSG:4326')
dalwin_marine_non_solar_polygons.set_crs('EPSG:4326')

dalwin_marine_solar_polygons['addr_housenumber'] = dalwin_marine_solar_polygons['region']
dalwin_marine_non_solar_polygons['addr_housenumber'] = dalwin_marine_non_solar_polygons['region']

In [505]:
comparison_pixels, all_polygons = filter_estate_pixels(dalwin_marine_solar_polygons, dalwin_marine_non_solar_polygons, cc30_2023_tuas_gdf)

Pixels CRS after conversion: EPSG:3857
Pixel geometry bounds: [11533327.48572822   134772.80710934 11540226.08181298   150634.29229034]
Initial polygons CRS: EPSG:4326
Polygons CRS after conversion: EPSG:3857
Polygon geometry bounds after CRS conversion: [11536115.60795974   147178.56209941 11536465.35153591   147549.23246591]
Number of intersecting pixels found: 104


c:\Users\teomi\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3508: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [506]:
plot_spatial_map(comparison_pixels, all_polygons, dalwin_marine_solar_polygons)

Index(['Unnamed: 0', 'Longitude', 'Latitude', 'SR_B1', 'SR_B2', 'SR_B3',
       'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_QA_AEROSOL', 'ST_B10',
       'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA',
       'ST_TRAD', 'ST_URAD', 'QA_PIXEL', 'QA_RADSAT', 'geometry', 'name',
       'time', 'ST_B10_Celsius', 'SR_B4_Scaled', 'SR_B5_Scaled', 'NDVI',
       'index_right', 'region', 'addr_housenumber'],
      dtype='object')
Index(['region', 'geometry', 'addr_housenumber'], dtype='object')
Index(['region', 'geometry', 'addr_housenumber'], dtype='object')
Average LST of new estate: 46.18257922532494
Average LST of old estate: 48.29989737117243


:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Polygons.I  :Polygons   [Longitude,Latitude]   (Average_Temp)
   .Polygons.II :Polygons   [Longitude,Latitude]

In [515]:
# import geopandas as gpd
# import hvplot.pandas  # ensure hvplot is called correctly
# import pandas as pd
# from shapely.geometry import box
# from bokeh.palettes import Oranges

# def plot_spatial_map_pixels(pixels_gdf, region_gdf):
#     # Convert region_gdf to the correct CRS for distance measurement in meters
#     region_gdf = region_gdf.to_crs('epsg:3857')
    
#     # Calculate the center of each polygon and create a buffer of 200 meters
#     region_gdf['center'] = region_gdf['geometry'].centroid
#     region_gdf['buffered_area'] = region_gdf['center'].buffer(1000)

#     # Filter pixels_gdf to only those within the buffered areas
#     pixels_gdf = pixels_gdf.to_crs('epsg:3857')

#     # Create 30m x 30m boxes around each point in pixels_gdf
#     half_width = 15  # half the width/height to create 30m x 30m boxes
#     pixels_gdf['geometry'] = pixels_gdf['geometry'].apply(lambda x: box(x.x - half_width, x.y - half_width, x.x + half_width, x.y + half_width))
    
#     # Check which pixels intersect with any of the buffered regions
#     intersects_any_buffer = lambda x: any(x.intersects(buff) for buff in region_gdf['buffered_area'])
#     pixels_gdf['intersects'] = pixels_gdf['geometry'].apply(intersects_any_buffer)
    
#     # Filter to pixels within the buffer
#     within_buffer_gdf = pixels_gdf[pixels_gdf['intersects']]
#     print("Number of pixels within 200 meters of the polygon centers:", len(within_buffer_gdf))
#     print(within_buffer_gdf)
    
#     # Extract min and max temperature values for color scaling
#     min_temp = within_buffer_gdf['ST_B10_Celsius'].min() if 'ST_B10_Celsius' in within_buffer_gdf.columns else 0
#     max_temp = within_buffer_gdf['ST_B10_Celsius'].max() if 'ST_B10_Celsius' in within_buffer_gdf.columns else 1

#     # Create the pixel plot with OpenStreetMap as the tile source
#     pixels_plot = within_buffer_gdf.hvplot.points('Longitude', 'Latitude', geo=True, c='ST_B10_Celsius', cmap=Oranges[256][::-1], frame_width=700, frame_height=500, colorbar=True, clim=(min_temp, max_temp), tiles='OSM', size=10)
#     squares_plot = within_buffer_gdf.hvplot.polygons('geometry', c='ST_B10_Celsius', cmap=Oranges[256][::-1], alpha=0.5, frame_width=700, frame_height=500, colorbar=True, clim=(min_temp, max_temp))

#     # Since the OpenStreetMap tiles are specified in pixels_plot, overlay squares_plot on pixels_plot
#     overlay_map = pixels_plot * squares_plot

#     return overlay_map

In [292]:
# plot_spatial_map_pixels(cc30_2023_tuas_gdf, tuas_bay_lane_solar_polygons)

Number of pixels within 200 meters of the polygon centers: 3590
         Unnamed: 0   Longitude  Latitude    SR_B1    SR_B2    SR_B3    SR_B4  \
3364237     1072925  103.633252  1.306779  10296.0  11195.0  12792.0  13432.0   
3364238     1072926  103.633522  1.306779   9852.0  10475.0  12077.0  12293.0   
3364239     1072927  103.633792  1.306780   9116.0   9750.0  11270.0  11555.0   
3364240     1072928  103.634061  1.306780   9297.0   9987.0  11511.0  11917.0   
3364241     1072929  103.634331  1.306780  10006.0  10983.0  12409.0  13063.0   
...             ...         ...       ...      ...      ...      ...      ...   
3370481     1190545  103.635419  1.288872   8674.0   9218.0  10279.0  10190.0   
3370482     1190546  103.635688  1.288872   9203.0  10037.0  11278.0  11043.0   
3370483     1190547  103.635958  1.288872   9567.0  10629.0  12175.0  12199.0   
3370484     1190548  103.636228  1.288872   8939.0   9691.0  11236.0  11399.0   
3370579     1192322  103.634071  1.288600   8

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Points.I   :Points   [Longitude,Latitude]   (ST_B10_Celsius)
   .Polygons.I :Polygons   [x,y]   (ST_B10_Celsius)

## Clementi Meadows

In [517]:
solar_clementi_meadows_gdf = get_blk_no_gdf_name('CLEMENTI MEADOWS', postal_gdf)

In [521]:
solar_clementi_meadows_polygons = get_polygons_gdf(solar_clementi_meadows_gdf, ['Clementi Avenue 4'], polygons_gdf)

In [525]:
solar_clementi_meadows_polygons = solar_clementi_meadows_polygons[~solar_clementi_meadows_polygons['addr_housenumber'].isin(['316', '315', '317', '317A', '318', '319', '320', '318A'])]

In [533]:
additional_solar_clementi_meadows_blk_no = ['301']
additional_solar_clementi_meadows_polygon = get_polygons_gdf_blk_no_list(additional_solar_clementi_meadows_blk_no, ['Clementi Avenue 4'], polygons_gdf)
solar_clementi_meadows_polygons = gpd.GeoDataFrame(pd.concat([solar_clementi_meadows_polygons, additional_solar_clementi_meadows_polygon]))

In [526]:
non_solar_clementi_meadows_blk_no = ['374', '373', '372', '371', '339', '341', '340', '342', '344', '381']

In [527]:
non_solar_clementi_meadows_polygons = get_polygons_gdf_blk_no_list(non_solar_clementi_meadows_blk_no, ['Clementi Avenue 4', 'Clementi Avenue 5'], polygons_gdf)

In [534]:
clementi_meadows_comparison_pixels, clementi_meadows_all_polygons = filter_estate_pixels(solar_clementi_meadows_polygons, non_solar_clementi_meadows_polygons, cc30_2023_clementi_gdf)

Pixels CRS after conversion: EPSG:3857
Pixel geometry bounds: [11549110.55818722   143813.57878176 11552261.17794055   149311.50607394]
Initial polygons CRS: EPSG:4326
Polygons CRS after conversion: EPSG:3857
Polygon geometry bounds after CRS conversion: [11550987.11269329   146731.07257885 11551683.47176794   147206.31030392]
Number of intersecting pixels found: 171


c:\Users\teomi\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3508: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [535]:
plot_spatial_map(clementi_meadows_comparison_pixels, clementi_meadows_all_polygons, solar_clementi_meadows_polygons)

Index(['Unnamed: 0', 'Longitude', 'Latitude', 'SR_B1', 'SR_B2', 'SR_B3',
       'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_QA_AEROSOL', 'ST_B10',
       'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA',
       'ST_TRAD', 'ST_URAD', 'QA_PIXEL', 'QA_RADSAT', 'geometry', 'name',
       'time', 'ST_B10_Celsius', 'SR_B4_Scaled', 'SR_B5_Scaled', 'NDVI',
       'index_right', 'addr_street', 'addr_housenumber'],
      dtype='object')
Index(['geometry', 'addr_street', 'addr_housenumber'], dtype='object')
Index(['geometry', 'addr_street', 'addr_housenumber'], dtype='object')
Average LST of new estate: 35.32107601485716
Average LST of old estate: 41.75427631580002


:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Polygons.I  :Polygons   [Longitude,Latitude]   (Average_Temp)
   .Polygons.II :Polygons   [Longitude,Latitude]

## Marine Drive

In [11]:
cc30_2023_marine_parade_gdf = cc30_2023_gdf[cc30_2023_gdf['name'] == 'MARINE PARADE']

In [13]:
mp_solar_blk_nos = ['60', '61', '62', '63', '64', '65', '66', '67']
mp_non_solar_blk_nos = ['73', '72', '71']
mp_solar_polygons = get_polygons_gdf_blk_no_list(mp_solar_blk_nos, ['Marine Drive'], polygons_gdf)
mp_non_solar_polygons = get_polygons_gdf_blk_no_list(mp_non_solar_blk_nos, ['Marine Drive'], polygons_gdf)

In [14]:
mp_comparison_pixels, mp_all_polygons = filter_estate_pixels(mp_solar_polygons, mp_non_solar_polygons, cc30_2023_marine_parade_gdf)

Pixels CRS after conversion: EPSG:3857
Pixel geometry bounds: [11563367.89558325   143033.86487355 11568259.74088133   146207.60741524]
Initial polygons CRS: EPSG:4326
Polygons CRS after conversion: EPSG:3857
Polygon geometry bounds after CRS conversion: [11567003.02631998   144929.83098516 11567220.14385483   145250.3470508 ]
Number of intersecting pixels found: 65


c:\Users\teomi\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3508: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [15]:
plot_spatial_map(mp_comparison_pixels, mp_all_polygons, mp_solar_polygons)

Index(['Unnamed: 0', 'Longitude', 'Latitude', 'SR_B1', 'SR_B2', 'SR_B3',
       'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_QA_AEROSOL', 'ST_B10',
       'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA',
       'ST_TRAD', 'ST_URAD', 'QA_PIXEL', 'QA_RADSAT', 'geometry', 'name',
       'time', 'ST_B10_Celsius', 'SR_B4_Scaled', 'SR_B5_Scaled', 'NDVI',
       'index_right', 'addr_street', 'addr_housenumber'],
      dtype='object')
Index(['geometry', 'addr_street', 'addr_housenumber'], dtype='object')
Index(['geometry', 'addr_street', 'addr_housenumber'], dtype='object')
Average LST of new estate: 41.06853649411609
Average LST of old estate: 42.32067641000002


:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Polygons.I  :Polygons   [Longitude,Latitude]   (Average_Temp)
   .Polygons.II :Polygons   [Longitude,Latitude]